<a href="https://colab.research.google.com/github/Scott-S-Lin/NTUT_PhD/blob/main/lab10_train_test_scott.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
import os

py_file_location = "/content/drive/MyDrive/Colab Notebooks"
sys.path.append(os.path.abspath(py_file_location)

In [ ]:
%pwd
%cd "/content/drive/MyDrive/Colab Notebooks"

In [ ]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from utils.models import create_vae_model
from utils.losses import reconstruction_loss
from utils.callbacks import SaveDecoderOutput, SaveDecoderModel
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


def parse_fn(dataset, input_size=(28, 28)):
    x = tf.cast(dataset['image'], tf.float32)
    x = tf.image.resize(x, input_size)
    x = x / 255.
    return x, x


dataset = 'mnist'     # 'cifar10', 'fashion_mnist', 'mnist'
log_dirs = 'logs_vae'
batch_size = 16
latent_dim = 2
input_shape = (28, 28, 1)

# Load datasets
train_data = tfds.load(dataset, split=tfds.Split.TRAIN)
test_data = tfds.load(dataset, split=tfds.Split.TEST)

# Setting datasets
AUTOTUNE = tf.data.experimental.AUTOTUNE  # 自動調整模式
train_data = train_data.shuffle(1000)
train_data = train_data.map(parse_fn, num_parallel_calls=AUTOTUNE)
train_data = train_data.batch(batch_size)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)
test_data = test_data.map(parse_fn, num_parallel_calls=AUTOTUNE)
test_data = test_data.batch(batch_size)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

# Callbacks function
model_dir = log_dirs + '/models'
os.makedirs(model_dir, exist_ok=True)
model_tb = keras.callbacks.TensorBoard(log_dir=log_dirs)
model_sdw = SaveDecoderModel(model_dir + '/best_model.h5', monitor='val_loss')
model_testd = SaveDecoderOutput(28, log_dir=log_dirs)

# create vae model
vae_model = create_vae_model(input_shape, latent_dim)

# training
optimizer = tf.keras.optimizers.RMSprop()
vae_model.compile(optimizer, loss=reconstruction_loss)
vae_model.fit(train_data, epochs=20, validation_data=test_data, callbacks=[model_tb, model_sdw, model_testd])


In [ ]:
%pwd

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


size = 28
n = 15
save_images = np.zeros((size * n, size * n, 1))
# linearly spaced coordinates corresponding to the 2D plot of digit classes in the latent space
grid_x = np.linspace(-1.5, 1.5, n)
grid_y = np.linspace(-1.5, 1.5, n)
model = tf.keras.models.load_model('logs_vae/models/best_model.h5')
for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        img = model(z_sample)
        save_images[i * size: (i + 1) * size, j * size: (j + 1) * size] = img.numpy()[0]

plt.imshow(save_images[..., 0], cmap='gray')
plt.show()
plt.imsave('output.png', save_images[..., 0], cmap='gray')

In [ ]:
%load_ext tensorboard

In [ ]:
tensorboard --logdir logs_vae